In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Check training device 

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Define model class

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print("Predicted class: {}".format(y_pred))

Predicted class: tensor([4], device='cuda:0')


In [19]:
print(pred_prob)

tensor([[0.0976, 0.0948, 0.0901, 0.0946, 0.1184, 0.1005, 0.0942, 0.1007, 0.1093,
         0.0998]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


## Check layers from scratch

In [20]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [23]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [24]:
layer1 = nn.Linear(in_features = 28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [27]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.5300, -0.6213,  0.3157,  0.1230, -0.4103,  0.0549, -0.3117,  0.0389,
          0.5993, -0.3498, -0.2849,  0.0925, -0.1944, -0.1240,  0.1154, -0.3375,
          0.0231, -0.3506, -0.0300, -0.1688],
        [ 0.4044, -0.5024,  0.2037, -0.2348, -0.0553, -0.0534, -0.3783,  0.3472,
         -0.0529, -0.0552,  0.1729,  0.0907, -0.4094,  0.0663,  0.2294, -0.1953,
         -0.0280, -0.2487,  0.1352, -0.3047],
        [ 0.5432, -0.4098,  0.0366, -0.3123,  0.0215, -0.0242, -0.5043,  0.2006,
          0.2823, -0.0910, -0.0789,  0.2236, -0.3800, -0.0541, -0.4321, -0.2870,
          0.1769, -0.4442,  0.1767, -0.5239]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.5300, 0.0000, 0.3157, 0.1230, 0.0000, 0.0549, 0.0000, 0.0389, 0.5993,
         0.0000, 0.0000, 0.0925, 0.0000, 0.0000, 0.1154, 0.0000, 0.0231, 0.0000,
         0.0000, 0.0000],
        [0.4044, 0.0000, 0.2037, 0.0000, 0.0000, 0.0000, 0.0000, 0.3472, 0.0000,
         0.0000, 0.1729, 0.0907, 0.0000, 0.0663, 0.22

In [30]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [33]:
softmax = nn.Softmax(dim=1)
pred_proba = softmax(logits)

In [34]:
pred_proba

tensor([[0.0962, 0.0883, 0.0925, 0.1094, 0.0816, 0.0962, 0.0803, 0.1288, 0.1134,
         0.1132],
        [0.1008, 0.0966, 0.0935, 0.1142, 0.0771, 0.0934, 0.0836, 0.1229, 0.1133,
         0.1046],
        [0.0974, 0.0924, 0.0882, 0.1195, 0.0777, 0.0929, 0.0827, 0.1255, 0.1090,
         0.1146]], grad_fn=<SoftmaxBackward0>)

##  Model parameter

In [35]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name}  \  Size: {param.size()}  |  Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight  \  Size: torch.Size([512, 784])  |  Values: tensor([[ 0.0172, -0.0192, -0.0126,  ..., -0.0269, -0.0278, -0.0022],
        [ 0.0099, -0.0019, -0.0189,  ...,  0.0230, -0.0266, -0.0012]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias  \  Size: torch.Size([512])  |  Values: tensor([0.0183, 0.0087], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight  \  Size: torch.Size([512, 512])  |  Values: tensor([[-0.0077, -0.0044, -0.0182,  ..., -0.0286,  0.0148,  0.0300],
        [-0.0170, -0.0343,  0.0417,  ...,  0.0344, -0.0023,  0.0160]],
       device='cuda:0', grad_f

In [40]:
next(model.named_parameters())

('linear_relu_stack.0.weight',
 Parameter containing:
 tensor([[ 0.0172, -0.0192, -0.0126,  ..., -0.0269, -0.0278, -0.0022],
         [ 0.0099, -0.0019, -0.0189,  ...,  0.0230, -0.0266, -0.0012],
         [ 0.0211,  0.0335, -0.0061,  ..., -0.0181,  0.0290,  0.0319],
         ...,
         [ 0.0242,  0.0067,  0.0040,  ...,  0.0295,  0.0302,  0.0236],
         [-0.0228,  0.0110,  0.0052,  ...,  0.0078,  0.0144,  0.0157],
         [-0.0231,  0.0144, -0.0256,  ..., -0.0018,  0.0142, -0.0113]],
        device='cuda:0', requires_grad=True))